<a href="https://colab.research.google.com/github/mish841/RAISE-26-Data-Science-AI-Competition-Submission/blob/mariam-phase2-progress/Data_Science_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

DATA_PATH = "dataset_A_news_full_10500.csv"
# df = pd.read_csv("dataset_A_news_full_10500.csv")


df = pd.read_csv(DATA_PATH)

df.head()
df.info()
df.isna().sum().sort_values(ascending=False).head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   title                       10500 non-null  object
 1   link                        10500 non-null  object
 2   date                        10500 non-null  object
 3   source                      10500 non-null  object
 4   number_of_characters_title  10500 non-null  int64 
 5   number_of_words_title       10500 non-null  int64 
 6   day_of_week                 10500 non-null  object
 7   month                       10500 non-null  object
 8   year                        10500 non-null  int64 
 9   quarter                     10500 non-null  int64 
 10  is_weekend                  10500 non-null  bool  
 11  classes_str                 10500 non-null  object
dtypes: bool(1), int64(4), object(7)
memory usage: 912.7+ KB


,0
title,0
link,0
date,0
source,0
number_of_characters_title,0
number_of_words_title,0
day_of_week,0
month,0
year,0
quarter,0
